Load some python libraries
The dependencies for this tutorials are
PIL, Numpy, Matplotlib



In [1]:
from __future__ import division, absolute_import, print_function
import os, sys, time, re, json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

imread = plt.imread
def imread8(im_file):
    ''' Read image as a 8-bit numpy array '''
    im = np.asarray(Image.open(im_file))
    return im

def read_png(res):
    img = Image.open(res)
    return np.asarray(img)

def read_npy(res):
    return np.load(res)

Connect to the game
===================
Load unrealcv python client, do :code:`pip install unrealcv` first.



In [2]:
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)

INFO:__init__:204:Got connection confirm: 'connected to NeighborhoodCV'


Make sure the connection works well



In [3]:
res = client.request('vget /unrealcv/status')
# The image resolution and port is configured in the config file.
print(res)

Is Listening
Client Connected
9000



Ground truth generation
=======================
Generate ground truth from this virtual scene



In [5]:
res = client.request('vget /camera/0/object_mask png')
object_mask = read_png(res)
res = client.request('vget /camera/0/normal png')
normal = read_png(res)

# Visualize the captured ground truth
plt.imshow(object_mask)
plt.figure()
plt.imshow(normal)

Get objects
======================
Write a json file with the object and their corresponding classes.



In [7]:
scene_objects = client.request('vget /objects').split(' ')
print('Number of objects in this scene:', len(scene_objects))

data = {}

for obj_id in scene_objects:
    obj_id_parts = obj_id.split('_')
    class_name = obj_id_parts[0]    
    data[obj_id] = class_name

# Write JSON file
import json
with open('neighborhood_classes.json', 'w') as outfile:
    json.dump(data, outfile)

Number of objects in this scene: 3762


Get object colors
======================
Map the objects to their labeling colors



In [ ]:
# TODO: replace this with a better implementation
class Color(object):
    ''' A utility class to parse color value '''
    regexp = re.compile('\(R=(.*),G=(.*),B=(.*),A=(.*)\)')
    def __init__(self, color_str):
        self.color_str = color_str
        match = self.regexp.match(color_str)
        (self.R, self.G, self.B, self.A) = [int(match.group(i)) for i in range(1,5)]

    def __repr__(self):
        return self.color_str

id2color = {} # Map from object id to the labeling color
for i, obj_id in enumerate(scene_objects):
    color = Color(client.request('vget /object/%s/color' % obj_id))
    id2color[obj_id] = color
    print('%d. %s : %s' % (i, obj_id, str(color)))

Number of objects in this scene: 3762
0. WorldSettings_1 : (R=0,G=0,B=0,A=255)
1. InstancedFoliageActor_0 : (R=0,G=0,B=255,A=255)
2. SkySphere : (R=0,G=255,B=0,A=255)
3. Landscape_0 : (R=0,G=255,B=255,A=255)
4. Road_2 : (R=255,G=0,B=0,A=255)
5. Road_3 : (R=255,G=0,B=255,A=255)
6. Road_5 : (R=255,G=255,B=0,A=255)
7. Road_7 : (R=255,G=255,B=255,A=255)
8. Road_9 : (R=0,G=0,B=127,A=255)
9. Road_10 : (R=0,G=255,B=127,A=255)
10. Road_11 : (R=255,G=0,B=127,A=255)
11. Road_12 : (R=255,G=255,B=127,A=255)
12. Road_13 : (R=0,G=127,B=0,A=255)
13. Road_14 : (R=0,G=127,B=255,A=255)
14. Road_15 : (R=255,G=127,B=0,A=255)
15. Road_17 : (R=255,G=127,B=255,A=255)
16. Road_18 : (R=0,G=127,B=127,A=255)
17. Road_19 : (R=255,G=127,B=127,A=255)
18. Road_20 : (R=127,G=0,B=0,A=255)
19. Road_21 : (R=127,G=0,B=255,A=255)
20. Road_22 : (R=127,G=255,B=0,A=255)
21. Road_23 : (R=127,G=255,B=255,A=255)
22. Road_24 : (R=127,G=0,B=127,A=255)
23. Road_25 : (R=127,G=255,B=127,A=255)
24. Road_26 : (R=127,G=127,B=0,A=255)
2

Parse the segmentation mask



In [18]:
# Matches a color from the object_mask and then returns that region color
def match_color(object_mask, target_color, tolerance=3):
    match_region = np.ones(object_mask.shape[0:2], dtype=bool)
    for c in range(3): # r,g,b
        min_val = target_color[c] - tolerance
        max_val = target_color[c] + tolerance
        channel_region = (object_mask[:,:,c] >= min_val) & (object_mask[:,:,c] <= max_val)
        match_region &= channel_region

    if match_region.sum() != 0:
        return match_region
    else:
        return None

id2mask = {}
for obj_id in scene_objects:
    print(obj_id)
    color = id2color[obj_id]
    mask = match_color(object_mask, [color.R, color.G, color.B], tolerance = 3)
    if mask is not None:
        id2mask[obj_id] = mask

# id2mask.keys() are all the matched objects
for idmask in id2mask.keys():
    print(idmask)

# This may take a while
# TODO: Need to find a faster implementation for this

WorldInfo_0
Carpet_5
SM_CoffeeTable_14
Couch_13
SM_Room_7
SM_Room_OuterShell_14
SM_Couch_1seat_5
Mug_22
EditorPlane_23
EditorPlane_24
EditorPlane_25
EditorPlane_26
EditorPlane_27
Mug_30
EditorPlane_31
EditorPlane_32
EditorPlane_33
EditorPlane_34
SlidingDoors_5
S_Trim_Floor_10
SM_Frame_15
SM_RoundCeilingLight_9
SM_RoundCeilingLight_10
SM_RoundCeilingLight_11
SM_RoundCeilingLight_12
SM_RoundCeilingLight_13
SM_RoundCeilingLight_14
SM_RoundCeilingLight_16
SM_RoundCeilingLight_17
SM_Railing_20
SM_Railing_21
SM_Railing_23
SM_Railing_24
SM_Railing_25
SM_Railing_26
SM_Railing_27
SM_Railing_28
SM_Railing_29
SM_Balcony_34
SM_Balcony_35
SM_Balcony_36
SM_Balcony_37
SM_Frame_38
SM_Frame_39
SM_Curtain_42
SM_Curtain_43
SM_Curtain_45
Statue_48
SM_FloorLamp_5
SM_FloorLamp_7
SM_FloorLamp_8
SM_TV_5
SM_DeskLamp_5
SM_Background_8
WallPiece1_22
WallPiece2_24
WallPiece3_26
WallPiece4_28
WallPiece5_30
WallPiece6_32
SM_Door_37
SM_Door_39
SM_Door_41
SM_Shelving_6
SM_Shelving_7
SM_Shelving_8
SM_Shelving_9
SM_She

Group the matched objects
===============

In [ ]:
class_groups = {}

# Go through the matched objects
for idmask in id2mask.keys():
    
    # Get the class from the data object we had earlier for json file
    curr_class = data[idmask]
    
    # If the class is not in the class_groups map, add it
    if curr_class not in class_groups:
        class_groups[curr_class] = [] 
        
    # Add the idmask to it's corresponding class
    class_groups[curr_class].append(idmask)

Begin normalization by class
=======

In [ ]:
for cls in class_groups.keys():
    

Print statistics of this virtual scene and this image
=====================================================
Load information of this scene



In [20]:
with open('object_category.json') as f:
    id2category = json.load(f)
categories = set(id2category.values())
# Show statistics of this frame
image_objects = id2mask.keys()
print('Number of objects in this image:', len(image_objects))
print('%20s : %s' % ('Category name', 'Object name'))
for category in categories:
    objects = [v for v in image_objects if id2category.get(v) == category]
    if len(objects) > 6: # Trim the list if too long
        objects[6:] = ['...']
    if len(objects) != 0:
        print('%20s : %s' % (category, objects))

Number of objects in this image: 123
       Category name : Object name
            Shelving : ['SM_Shelving_7', 'SM_Shelving_6', 'SM_Shelving_9', 'SM_Shelving_8']
                Bowl : ['SM_Bowl_29']
               Couch : ['SM_Couch_1seat_5', 'Couch_13']
                Book : ['BookLP_139', 'BookLP_153', 'BookLP_134', 'BookLP_135', 'BookLP_136', 'BookLP_137', '...']
            DeskLamp : ['SM_DeskLamp_5']
     CoatHookBacking : ['CoatHookBacking_7']
               Plant : ['SM_Plant_8']
                Door : ['SM_Door_39']
          Trim_Floor : ['S_Trim_Floor_10']
                Vase : ['SM_Vase_22', 'SM_Vase_21', 'SM_Vase_20', 'SM_Vase_18', 'SM_Vase_16', 'SM_Vase_17']
              Carpet : ['Carpet_7']
           FloorLamp : ['SM_FloorLamp_7']
              Switch : ['Switch_7']
               Frame : ['SM_Frame_39']
           WallPiece : ['WallPiece6_32', 'WallPiece2_24', 'WallPiece1_22', 'WallPiece3_26']
            CoatHook : ['CoatHook_17', 'CoatHook_16']


Show the annotation color of some objects



In [ ]:
ids = ['SM_Couch_1seat_5', 'SM_Vase_17', 'SM_Shelving_6', 'SM_Plant_8']
# for obj_id in ids:
obj_id = ids[0]
color = id2color[obj_id]
print('%s : %s' % (obj_id, str(color)))
# color_block = np.zeros((100,100, 3)) + np.array([color.R, color.G, color.B]) / 255.0
# plt.figure(); plt.imshow(color_block); plt.title(obj_id)

Plot only one object



In [ ]:
mask = id2mask['SM_Plant_8']
plt.figure(); plt.imshow(mask)

Show all sofas in this image



In [ ]:
couch_instance = [v for v in image_objects if id2category.get(v) == 'Couch']
mask = sum(id2mask[v] for v in couch_instance)
plt.figure(); plt.imshow(mask)

Change the annotation color, fixed in v0.3.9
You can use this to make objects you don't care the same color



In [ ]:
client.request('vset /object/SM_Couch_1seat_5/color 255 0 0') # Change to pure red
client.request('vget /object/SM_Couch_1seat_5/color')
res = client.request('vget /camera/0/object_mask png')
object_mask = read_png(res)
plt.imshow(object_mask)

Clean up resources
==================



In [ ]:
client.disconnect()